<font color='blue'><h1>Introdução Prática ao PyTorch</h1></font>

<font color='0471A6'><h2>Loop de Otimização no PyTorch - Exemplo com Classificação Multi-classe</h2></font>

- O modelo é treinado / otimizado em um loop de otimização
- Cada iteração será uma época
- Cada época contém duas partes principais:
  - Loop de treinamento
    - iteração sobre o conjunto de treinamento
  - Loop de validação
    - iteração sobre o conjunto de validação

Hiperparâmetros (parâmetros ajustáveis pelo projetista). Exemplos:
- Número de Épocas
- Tamanho do Lote (Batch)
- Taxa de Aprendizagem

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
iris = load_iris()
X, y = iris.data, iris.target

X_train, X_test, y_train, y_test\
= train_test_split(X, y,\
test_size=0.25, random_state=42)

In [3]:
#scaler = StandardScaler()
#X_train = scaler.fit_transform(X_train)
#X_test = scaler.transform(X_test)

In [4]:
X_train_tensor = torch.tensor(X_train, dtype=torch.float)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

In [5]:
# definindo o conjunto de dados com tensores
train_dataset = TensorDataset(
    X_train_tensor, y_train_tensor)

# definindo o dataloader
train_dataloader = DataLoader(
    train_dataset, batch_size=10, shuffle=True)

In [6]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(4, 10)
        self.fc2 = nn.Linear(10, 3)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.softmax(x)
        return x

net = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01)

In [7]:
net.train()

# treinando a rede neural
for epoch in range(100):
    running_loss = 0.0
    for i, data in enumerate(train_dataloader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    if epoch % 10 == 0:
      print('Epoch %d loss: %.3f' % (epoch, 
                                     running_loss / len(train_dataloader)))

Epoch 0 loss: 1.137
Epoch 10 loss: 1.113
Epoch 20 loss: 1.102
Epoch 30 loss: 1.072
Epoch 40 loss: 1.079
Epoch 50 loss: 1.063
Epoch 60 loss: 1.055
Epoch 70 loss: 1.053
Epoch 80 loss: 1.024
Epoch 90 loss: 0.987


In [8]:
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=True)

correct = 0
total = 0
net.eval()
with torch.no_grad():
    for data in test_dataloader:
        inputs, labels = data
        outputs = net(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += 1
        correct += (predicted == labels).sum().item()

print('Accuracy: %.4f %%' % (100 * correct / total))

Accuracy: 71.0526 %


# Referências 

* Documentação do Pytorch. Disponível em https://pytorch.org 
* Tutorial oficial do Pytorch. Disponível em https://pytorch.org/tutorials/beginner/basics/intro.html 
* Introdução ao Pytorch. Canal “Programação Dinâmica”. Disponível em https://www.youtube.com/watch?v=cGxv8tOaA7I 
* PyTorch Crash Course - Getting Started with Deep Learning. Canal AssemblyAI". Disponível em https://youtu.be/OIenNRt2bjg 
